Execute a célula abaixo antes de iniciar, pois ela instalará uma dependência obrigatória para a execução do código de extração do texto selecionado e comentários do(s) PDF(s).

In [1]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 72.7 MB/s eta 0:00:00


In [29]:
import fitz
import io
from tabulate import tabulate
from google.colab import auth
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload,MediaFileUpload

Na célula seguinte, você deve colocar o id da pasta do Google Drive onde estão os arquivos PDFs. Note que o id é o valor que aparece na URL: https://drive.google.com/drive/folders/ID

In [5]:
drive_id = ""

In [40]:
escopo = [
        "https://www.googleapis.com/auth/drive.appdata",
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive"
    ]
auth.authenticate_user()
creds, _ = google.auth.default(scopes=escopo)

servico_drive = build("drive", "v3", credentials=creds)

In [20]:
# Retorna uma tabela em formatação markdown com os textos em highlight, bem como suas respectivas páginas e comentários
def fichamento(pdf: fitz.Document):
    # Criar uma tabela com os textos em highlight, bem como suas respectivas páginas e comentários
    texto_highlight = []
    for numero_pagina in range(len(pdf)): # numero_pagina pode não ser o número real da página do PDF, dependendo de como está formatado
        pagina = pdf.load_page(numero_pagina)
        anotacoes = pagina.annots()
        if anotacoes:
            for annot in anotacoes:
                if annot.type[1] == 'Highlight':
                    caixa_conteudo = annot.rect
                    palavras = pagina.get_text("words")
                    # Conferir palavras que estão dentro do box de highlight
                    palavras_highlight = ""
                    for palavra in palavras:
                        caixa_palavra = fitz.Rect(palavra[:4])
                        if caixa_palavra.intersects(caixa_conteudo):
                            palavras_highlight += palavra[4] + " "

                    texto_highlight.append({
                        "Página do PDF": pagina.number + 1, # a primeira página é considerada 0, por isso somar +1 para saber a página correta do PDF
                        "Texto": palavras_highlight.strip(),
                        "Comentário": annot.info.get("content")
                    })
    return tabulate(texto_highlight, headers="keys", tablefmt="pipe")

In [27]:
# Retorna os arquivos ou pastas encontradas na pasta de ID id_pasta
def pesquisar_arquivos_pasta(id_pasta: str):
  arquivos = []

  query = f"'{id_pasta}' in parents"

  resposta_api = (
      servico_drive.files().list(
          q=query,
          fields="files(id, name, mimeType)",
          supportsAllDrives=True,
          includeItemsFromAllDrives=True
      ).execute()
  )

  for item in resposta_api.get("files", []):
    if item['mimeType'] == 'application/vnd.google-apps.folder':
      arquivos.extend(pesquisar_arquivos_pasta(item['id']))
    else:
      arquivos.append(item)

  return arquivos

In [30]:
arquivos = pesquisar_arquivos_pasta(drive_id)
if not arquivos:
  print("Nenhum arquivo encontrado na pasta do Drive.")
else:
  print(f"{len(arquivos)} arquivos encontrados.")

42 arquivos encontrados.
[{'id': '1a574WJlWFZ1maxs1jZ_bx9OvwJAqjajtDOr99MAXnqI', 'name': 'Fichamentos', 'mimeType': 'application/vnd.google-apps.document'}, {'id': '16XZxYcwkhGtRk9HfKgR4gwzsy2k789pP', 'name': 'fichamento.md', 'mimeType': 'text/markdown'}, {'id': '17O9U3ZxDSDMyrmOeiFPklZV2zoT2tjH5', 'name': 'OK SIMIÃO - Crise climática, mecanismos de mercado e a financeirização da natureza.pdf', 'mimeType': 'application/pdf'}, {'id': '13m9RuriGTVbDhXTRW_sNJ7sNopgE8thm', 'name': 'OK ZACARIAS - A lógica destrutiva do capital crise ambiental e mudanças climáticas.pdf', 'mimeType': 'application/pdf'}, {'id': '1haALoiEf2OV35O_zAJwJmss9MXdeNdyA', 'name': 'OK SILVA - Corpos antropofágicos - Supermáquina e interseccionalidades em cartoescrita de fluxos indisciplinares.pdf', 'mimeType': 'application/pdf'}, {'id': '1EzNGilYv6OCkAcz7Yu4q0PiadwH8XDl8', 'name': 'OK PEREIRA - Estudos Multiespécies.pdf', 'mimeType': 'application/pdf'}, {'id': '1BxXZHOmn3-3yhjFDTtxbJ0NhEiATGV3p', 'name': 'OK REIS - O e

In [31]:
for arquivo in arquivos:
  if arquivo['mimeType'] == 'application/pdf':
    print(f"\tTentando baixar o {arquivo['name']}")

    # Baixar arquivo do Drive como I/O
    request = servico_drive.files().get_media(fileId=arquivo['id'])
    file = io.BytesIO()
    downloader = MediaIoBaseDownload(file, request)
    done = False
    while done is False:
      status, done = downloader.next_chunk()

      # Checar se o arquivo é um PDF válido e fazer o fichamento
      try:
        with fitz.open(stream=file, filetype="pdf") as pdf:
          print(f"\t\tArquivo válido como PDF, iniciando extração para fichamento...")
          tabela = fichamento(pdf)

          # Abrir o arquivo markdown para salvar o título e tabela
          try:
            with open("/content/fichamento.md", "a", encoding="utf-8") as output:
              titulo = ""
              # Essa regra confere se o nome do arquivo está na formatação "OK Autoria - Título.pdf"
              if arquivo['name'].startswith("OK "):
                titulo = arquivo['name'][3:-4]
              else:
                titulo = arquivo['name'][:-4]
              output.write(f"\n# {titulo} \n")
              output.write(tabela)
              print("\t\tSalvou com sucesso o fichamento desse PDF no arquivo markdown!")
          except Exception as error:
            print(f"\t\tOcorreu um erro na hora de salvar o fichamento, tente novamente")
            print(error)
      except Exception as error:
        print(f"\t\tPDF inválido: {arquivo['name']}")
        print(error)

	Tentando baixar o OK SIMIÃO - Crise climática, mecanismos de mercado e a financeirização da natureza.pdf (17O9U3ZxDSDMyrmOeiFPklZV2zoT2tjH5)
		O OK SIMIÃO - Crise climática, mecanismos de mercado e a financeirização da natureza.pdf é válido como PDF, iniciando extração para fichamento...
		Salvou com sucesso o fichamento no arquivo markdown!
	Tentando baixar o OK ZACARIAS - A lógica destrutiva do capital crise ambiental e mudanças climáticas.pdf (13m9RuriGTVbDhXTRW_sNJ7sNopgE8thm)
		O OK ZACARIAS - A lógica destrutiva do capital crise ambiental e mudanças climáticas.pdf é válido como PDF, iniciando extração para fichamento...
		Salvou com sucesso o fichamento no arquivo markdown!
	Tentando baixar o OK SILVA - Corpos antropofágicos - Supermáquina e interseccionalidades em cartoescrita de fluxos indisciplinares.pdf (1haALoiEf2OV35O_zAJwJmss9MXdeNdyA)
		O OK SILVA - Corpos antropofágicos - Supermáquina e interseccionalidades em cartoescrita de fluxos indisciplinares.pdf é válido como PDF

No próximo passo, o arquivo **fichamento.md** que está localmente salvo na seção "Arquivos" (ver ícone de pasta no menu lateral esquerdo), será importado já em formato Google Docs na pasta informada anteriormente.

In [45]:
# Salvar o arquivo markdown dos fichamentos no Google Drive
metadados_markdown = {
    "name": "Fichamentos - Ferramenta de extração de PDF",
    "mimeType": "application/vnd.google-apps.documents",
    "parents": [
        f"{drive_id}"
        ]
    }
midia_markdown = MediaFileUpload("/content/fichamento.md", mimetype="text/markdown")
upload_arquivo_fichamento = servico_drive.files().create(
    body=metadados_markdown,
    media_body=midia_markdown,
    supportsAllDrives=True,
    fields="id"
    ).execute()

print(f'Fichamento salvo no Google Drive: {upload_arquivo_fichamento.get("id")} ! Boa escrita!')


Fichamento salvo no Google Drive: 1dJFVHehVFEv52kOuqINKyesFg1ScGBPCoricHLR6qbI
